## <b> [3단계] AI 모델링
---

## 1, 2 단계에서 진행하신 내용을 아래에 그대로 진행해주세요.

---

# <b>Step 0. 라이브러리 import 및 데이터 불러오기
### **가. 라이브러리 import**

* 데이터 프레임 관련 라이브러리

In [1]:
#라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

### **나.  학습데이터 불러오기**
- label 0, 1 변경
- train 데이터의 결측치 2개 행 삭제

In [2]:
train_df = pd.read_csv('data/train.csv')
test_x = pd.read_csv('data/test.csv')
train_df.shape,train_df.shape, test_x.shape

((3664, 24), (3664, 24), (2441, 24))

In [3]:
train_df = train_df.dropna()
train_df.isnull().sum()

url_len                    0
url_num_hyphens_dom        0
url_path_len               0
url_domain_len             0
url_hostname_len           0
url_num_dots               0
url_num_underscores        0
url_query_len              0
url_num_query_para         0
url_ip_present             0
url_entropy                0
url_chinese_present        0
url_port                   0
html_num_tags('iframe')    0
html_num_tags('script')    0
html_num_tags('embed')     0
html_num_tags('object')    0
html_num_tags('div')       0
html_num_tags('head')      0
html_num_tags('body')      0
html_num_tags('form')      0
html_num_tags('a')         0
html_num_tags('applet')    0
label                      0
dtype: int64

In [4]:
train_df.shape

(3662, 24)

In [5]:
train_df['label'] = train_df['label'].replace({'benign' : 0, 'malicious' : 1})

### **다.  데이터 전처리**
- 값이 0밖에 없는 특징 제거
    1. url_chinese_present
    2. html_num_tags('applet')
- 상관관계가 높은 변수 제거
    - url_hostname_len 과 url_domain_len : 0.998 -> label과 상관관계가 더 높은 url_hostname_len 만 살리기
    - url_len 과 url_path_len : 0.944 -> label과 상관관계가 더 높은 url_path_len 만 살리기
    - url_query_len_is 와 url_num_query_para_is : 0.914 : label과 상관관계가 더 높은 url_query_len_is 만 살리기
- 중복 데이터 제거
- test 데이터 결측치 -> 중앙값으로 대체

#### 값이 0밖에 없는 특징 제거
1. url_chinese_present
2. html_num_tags('applet')

In [6]:
train_df = train_df.drop(["url_chinese_present", "html_num_tags('applet')"], axis=1)
test_x = test_x.drop(["url_chinese_present", "html_num_tags('applet')"], axis=1)

#### 상관관계가 높은 변수 제거
- url_hostname_len 과 url_domain_len : 0.998 -> label과 상관관계가 더 높은 url_hostname_len 만 살리기
- url_len 과 url_path_len : 0.944 -> label과 상관관계가 더 높은 url_path_len 만 살리기
- url_query_len_is 와 url_num_query_para_is : 0.914 : label과 상관관계가 더 높은 url_query_len_is 만 살리기

In [7]:
train_df = train_df.drop(["url_domain_len", "url_path_len", "url_num_query_para"], axis=1)
test_x = test_x.drop(["url_domain_len", "url_path_len", "url_num_query_para"], axis=1)

#### 중복 데이터 제거

In [8]:
train_df = train_df.drop_duplicates()

#### test 데이터 결측치 처리
- 평균으로 대체

In [9]:
test_x.isnull().sum()

id                           0
url_len                      0
url_num_hyphens_dom          0
url_hostname_len           463
url_num_dots                 0
url_num_underscores          0
url_query_len                0
url_ip_present               0
url_entropy                  0
url_port                     0
html_num_tags('iframe')      0
html_num_tags('script')    467
html_num_tags('embed')       0
html_num_tags('object')      0
html_num_tags('div')         0
html_num_tags('head')        0
html_num_tags('body')        0
html_num_tags('form')      467
html_num_tags('a')         461
dtype: int64

In [10]:
null_cols = ['url_hostname_len', "html_num_tags('script')", "html_num_tags('form')", "html_num_tags('a')"]
for col in null_cols:
    value = train_df[col].median()
    test_x[col] = test_x[col].fillna(value)

In [11]:
test_x.isnull().sum()

id                         0
url_len                    0
url_num_hyphens_dom        0
url_hostname_len           0
url_num_dots               0
url_num_underscores        0
url_query_len              0
url_ip_present             0
url_entropy                0
url_port                   0
html_num_tags('iframe')    0
html_num_tags('script')    0
html_num_tags('embed')     0
html_num_tags('object')    0
html_num_tags('div')       0
html_num_tags('head')      0
html_num_tags('body')      0
html_num_tags('form')      0
html_num_tags('a')         0
dtype: int64

### **마. Confusion Matrix 함수 정의**


#### Confusion Matrix란 Training 을 통한 Prediction 성능을 측정하기 위해 예측 value와 실제 value를 비교하기 위한 표입니다.
#### 아래 함수는 이번 과제에서 confusion matrix 결과를 보기 쉽게 표현한 것으로 사용 예를 참고하여 모델 결과 확인에 사용하시기 바랍니다.



**<span style="color:green">[참고링크] 공식 Document**</span>
 
* confusion matrix(https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [12]:
from sklearn.metrics import classification_report as creport
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [13]:
def plot_confusion_matrix(ax, matrix, labels = ['malicious','benign'], title='Confusion matrix', fontsize=9):
    ax.set_xticks([x for x in range(len(labels))])
    ax.set_yticks([y for y in range(len(labels))])

    # Place labels on minor ticks
    ax.set_xticks([x + 0.5 for x in range(len(labels))], minor=True)
    ax.set_xticklabels(labels, rotation=90, fontsize=fontsize, minor=True)
    ax.set_yticks([y + 0.5 for y in range(len(labels))], minor=True)
    ax.set_yticklabels(labels[::-1], fontsize=fontsize, minor=True)

    # Hide major tick labels
    ax.tick_params(which='major', labelbottom='off', labelleft='off')

    # Finally, hide minor tick marks
    ax.tick_params(which='minor', width=0)

    # Plot heat map
    proportions = [1. * row / sum(row) for row in matrix]
    ax.pcolor(np.array(proportions[::-1]), cmap=plt.cm.Blues)

    # Plot counts as text
    for row in range(len(matrix)):
        for col in range(len(matrix[row])):
            confusion = matrix[::-1][row][col]
            if confusion != 0:
                ax.text(col + 0.5, row + 0.5, int(confusion),
                        fontsize=fontsize,
                        horizontalalignment='center',
                        verticalalignment='center')

    # Add finishing touches
    ax.grid(True, linestyle=':')
    ax.set_title(title, fontsize=fontsize)
    ax.set_xlabel('prediction', fontsize=fontsize)
    ax.set_ylabel('actual', fontsize=fontsize)

    plt.show()

### <span style="color:blue">[예시] Confusion Matrix 사용 방법<span>

- 샘플
#### > confusion = confusion_matrix(test_y, dt_pred)
#### > fig, ax = plt.subplots(figsize=(10,3))
#### > plot_confusion_matrix(ax, confusion, fontsize=30)


---

## <b> Q. AI 분류 모델 만들기

* 전처리한 데이터셋을 활용해 악성사이트 여부를 판별하는 AI 분류 모델을 <font color="red">최소 3개 이상 </font>만들어보세요.

In [14]:
train_x = train_df.drop("label", axis=1)
train_y = train_df.loc[:, "label"]
feature_names = list(train_x)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score

### XGBoost

In [16]:
from xgboost import XGBClassifier

In [20]:
model = XGBClassifier(
    learning_rate=0.01,
    n_estimators=1000,
    max_depth=9,
    min_child_weight=1,
    gamma=0.1,
    colsample_bytree=0.6,
    subsample=0.74
)

In [21]:
model.fit(train_x, train_y)
test = test_x.drop('id', axis=1)
y_pred = model.predict(test)

In [22]:
# 'benign' : 0, 'malicious' : 1
prediction = np.where(y_pred == 0, 'benign', 'malicious')

In [23]:
pd.DataFrame(data = {"id" : test_x['id'], "label" : prediction}, columns=['id', 'label']).to_csv("prediction_11_최적_전처리.csv", index=False)